In [1]:
import re, requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from gamelogs_to_mongo import get_return_dates
from pymongo import MongoClient
from data_cleaning import *
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle('../data/df_inj.pkl')
df = df[(df['Date'] >= '1994-07-01')&df['New_Inj']]
df

,Date,player,Team,Status,Notes,bbref_id,from,to,height,weight,...,ppg_career_b4,rpg_career_b4,apg_career_b4,pmg_career_b4,League_Years,Num_Inj_Career,Num_Inj_Season,POS_G,POS_F,POS_C
1597,1994-07-06,Patrick Ewing,Knicks,Injured,arthroscopic surgery on knee (out 6-8 weeks),ewingpa01,1986,2002,84,240,...,23.73,10.39,2.15,NaN,8,4,1,0,1,1
1598,1994-08-16,Brent Price,Bullets/Wizards,Injured,surgery on knee to repair torn ACL (out for se...,pricebr01,1993,2002,73,165,...,4.98,1.45,2.76,NaN,2,1,1,1,0,0
1599,1994-08-24,Doug Christie,Lakers,Injured,surgery on left ankle to remove calcium deposi...,chrisdo01,1993,2007,78,200,...,8.85,3.12,2.1,NaN,2,1,1,1,1,0
1600,1994-08-24,John Battle,Cavaliers,Injured,surgery on left knee to remove loose cartilage...,battljo01,1986,1995,74,175,...,8.74,1.39,2.04,NaN,9,2,1,1,0,0
1601,1994-09-15,LaPhonso Ellis,Nuggets,Injured,stress fracture in right kneecap (out 8 weeks),ellisla01,1993,2003,80,240,...,15.01,8.8,1.99,NaN,2,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62218,2021-06-06,James Harden,Nets,Injured,placed on IL with tightness in right hamstring,hardeja01,2010,2021,77,220,...,24.92,5.46,6.45,3.82,11,27,7,1,0,0
62219,2021-06-07,Jordan Nwora,Bucks,Injured,placed on IL with left thigh injury,nworajo01,2021,2021,80,225,...,5.5,1.94,0.22,0.03,0,2,2,0,1,0
62221,2021-06-08,Mike Conley,Jazz,Injured,placed on IL with strained right hamstring,conlemi01,2008,2021,73,175,...,15.07,3.04,5.74,1.95,13,46,9,1,0,0
62223,2021-06-09,Sam Merrill,Bucks,Injured,placed on IL with sprained right ankle,merrisa01,2021,2021,76,205,...,2.71,0.91,0.65,-1.15,0,2,2,1,0,0


In [7]:
nsc = df.Notes.str.contains

In [21]:
df[nsc('(^pla.*?n I\w)')
    &nsc('( \(.*?\))')
    &~nsc('(date', regex=False)
    &~nsc('(out', regex=False)
    &~nsc('dropped')
    &~nsc('(S)')
    ].Notes.str.extract('( \(.*?\))')[0].unique()

array([' (S)', ' (Stats)', ' (suffered in car accident)',
       ' (stomach flu)', ' (per Stats)',
       ' (and dropped from playoff roster)',
       ' (dropped from playoff roster)', ' (lower back)',
       ' (sore left knee per P)', ' (right foot per Stats)',
       ' (back spasms per Stats)', ' (back per Stats)', ' (P)',
       ' (lower back spasms per Stats)', ' (left shoulder per Stats)',
       ' (plantar fasciatis)', ' (hamstring tendinitis per Stats)',
       ' (pinky finger per Stats)', ' (sore left shoulder per P)',
       ' (bruised left bicep per P)', ' (groin)',
       ' (per Stats & Bulls media guide)', ' (back surgery per Stats)',
       ' (shin splints)', ' (knee injury per ESPN)',
       ' (per Bulls media guide)',
       ' (player had not officially submitted retirement papers)',
       ' (right knee tendinitis per E)', ' (soreness)', ' (DNP)',
       ' (left?)', ' (sprained right ankle per P)',
       ' (torn per Bulls media guide)',
       ' (sprained right ankle p

In [56]:
## Regex commands
per_source = r'(\(| )(per .*?)\)' 
date_approx = r'( \(date .*?\))'
out_for_x = r'( \(ou)(t|r)( .*?\))'
# df[nsc(per_source)].Notes.apply(lambda x: re.sub(per_source, ')', x))
# df[nsc(date_approx)].Notes.apply(lambda x: re.sub(date_approx, '', x))
# df[nsc(out_for_x)].Notes.apply(lambda x: re.sub(out_for_x, '', x))
df.Notes.apply(lambda x: re.sub(per_source, ')', x)).apply(lambda x: re.sub(date_approx, '', x)).apply(lambda x: re.sub(out_for_x, '', x)).str.extract('( \(.*?\))')[0].unique()

array([nan, ' (DTD)', ' (S)', ' (DNP)', ' (left game)', ' (day-to-day)',
       ' (Stats)', ' (suffered in car accident)', ' (left knee)',
       ' (stomach flu)', ' (and dropped from playoff roster)',
       ' (dropped from playoff roster)', ' (roughening under kneecap)',
       ' (lower back)', ' (sore left knee)', ' (car accident)',
       ' (eye socket)', ' (right foot)', ' (bruised)', ' (back spasms)',
       ' (back)', ' (P)', ' (lower back spasms)', ' (left shoulder)',
       ' (plantar fasciatis)', ' (torn)', ' (stepped on nail)', ' (knee)',
       ' (likely out for season)', ' (sprained)', ' (elbow)',
       ' (right ankle)', ' (left thumb)', ' (right thumb)',
       ' (sore Achilles)', ' (bruised ribs)', ' (hamstring tendinitis)',
       ' (strained left quadricep)', ' (not with team)',
       ' (left and right ankles)', ' (pinky finger)',
       ' (missed practice)', ' (missed pre-season games)',
       ' (sore left shoulder)', ' (bruised left bicep)', ' (groin)',
       ' (